In [4]:
'''
Peak hours, seasonality trends. 

Areas to interrogate: 
- How has the distribution of rideable types changed over time? 
- When did e-bikes get added to the system? 
- Is there a correlation between day of the week and ride length? bike type and day of the week or hours of the day? 
- Is there a correlation between day of the week and bike type? 
- Station imbalance: Which stations consistently run empty or full?
- Bike redistribution needs: Where do bikes accumulate vs where they're needed?
- Peak capacity stress: Which stations/times show highest demand relative to supply?
- Rush hour vs leisure patterns: Different station pairs popular at different times?
- Weekend vs weekday geographic preferences
- Holiday/event impact on usage patterns
'''

"\nPeak hours, seasonality trends. \n\nAreas to interrogate: \n- How has the distribution of rideable types changed over time? \n- When did e-bikes get added to the system? \n- Is there a correlation between day of the week and ride length? bike type and day of the week or hours of the day? \n- Is there a correlation between day of the week and bike type? \n- Station imbalance: Which stations consistently run empty or full?\n- Bike redistribution needs: Where do bikes accumulate vs where they're needed?\n- Peak capacity stress: Which stations/times show highest demand relative to supply?\n- Rush hour vs leisure patterns: Different station pairs popular at different times?\n- Weekend vs weekday geographic preferences\n- Holiday/event impact on usage patterns\n"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# plotting-- for future use
plt.style.use('default')
sns.set_palette("husl")

# loading
df_enhanced = pd.read_parquet('../data/processed/cleaned_citibike_data_with_features.parquet')

# check for correct load
print(f"Data shape: {df_enhanced.shape}")
print(df_enhanced.head())

Data shape: (2115989, 17)
            ride_id  rideable_type              started_at  \
0  B2B980E0EAE1D6F1   classic_bike 2024-02-25 20:25:40.894   
1  1069DDA1FED20568   classic_bike 2024-02-21 22:21:38.446   
2  B58850AF6F2D8BD5  electric_bike 2024-02-14 08:31:14.609   
3  D46E6C5A69048E11  electric_bike 2024-02-05 08:42:25.999   
4  707AF4CF2C7834C2  electric_bike 2024-02-08 11:13:15.969   

                 ended_at        start_station_name start_station_id  \
0 2024-02-25 20:43:58.504  Greenwich St & Hubert St          5470.10   
1 2024-02-21 22:40:12.259  Greenwich St & Hubert St          5470.10   
2 2024-02-14 08:42:30.427   Mercer St & Bleecker St          5679.05   
3 2024-02-05 08:56:26.899          E 20 St & FDR Dr          5886.13   
4 2024-02-08 11:18:44.259          E 20 St & FDR Dr          5886.13   

          end_station_name end_station_id  start_lat  start_lng    end_lat  \
0  Hudson Blvd W & W 36 St        6611.07  40.721319 -74.010065  40.756765   
1  Hudson Bl

In [3]:
column_names = df_enhanced.columns.tolist()

# Print the list of column names
print(column_names)

['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual', 'date', 'hour', 'day_of_week', 'duration_minutes']
